In [6]:
from stage import getStage
import pandas as pd

In [7]:
R3000TXT = open("stockData/russell3000Symbols.txt", "r")
R3000List = R3000TXT.read()
dfR3000Tickers = R3000List.split(",")
R3000TXT.close()
dfR3000 = list(zip(dfR3000Tickers, ['not hold']*len(dfR3000Tickers)))

In [8]:
dfSPYTickers = pd.read_csv('stockData/S&P500Symbols.csv')
dfSPY = list(zip(dfSPYTickers['Symbol'], ['not hold']*len(dfR3000Tickers)))

In [9]:
dfReal = dfR3000+dfSPY

In [10]:
# Open a file
result = open("result.txt", "w")
for symbol in dfReal:
    try:
        df = getStage(symbol[0],symbol[1])
        shares = 0
        sharesSPY = 0
        if df.iloc[-1].Stage=="Stage 2":
            if df.iloc[-2].Stage!="Stage 2":
                print(symbol[0]+" buy")
                result.write(symbol[0]+" buy\n")
            else:
                print(symbol[0]+" hold")
                result.write(symbol[0]+" hold\n")
        else:
            if df.iloc[-2].Stage=="Stage 2":
                print(symbol[0]+" sell")
                result.write(symbol[0]+" sell\n")
    except:
        print(symbol[0]+" Stock Not Found or too expensive")
# Close opend file
result.close()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- FUBC: No data found for this date range, symbol may be delisted
FUBC Stock Not Found or too expensive
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%**********************